# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
R. Hoppe  ->  R. Hoppe  |  ['R. Hoppe']
M. Bergemann  ->  M. Bergemann  |  ['M. Bergemann']
P. Eitner  ->  P. Eitner  |  ['P. Eitner']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
S. Jiao  ->  S. Jiao  |  ['S. Jiao']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
J. d. Brok  ->  J. d. Brok  |  ['J. d. Brok']
F. Xu  ->  F. Xu  |  ['F. Xu']
F. Walter  ->  F. Walter  |  ['F. Walter']


D. Gasman  ->  D. Gasman  |  ['D. Gasman']
T. Henning  ->  T. Henning  |  ['T. Henning']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']
E. Matthews  ->  E. Matthews  |  ['E. Matthews']
P. Sudarshan  ->  P. Sudarshan  |  ['P. Sudarshan']
M. Flock  ->  M. Flock  |  ['M. Flock']
D. M. Fuksman  ->  D. M. Fuksman  |  ['D. M. Fuksman']
Arxiv has 83 new papers today
          10 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/10 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2511.13828


extracting tarball to tmp_2511.13828... done.


L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']


Error retrieving bib data for madau2014sfrd. The best fit to our data is the orange dashed line: 'madau2014sfrd. the best fit to our data is the orange dashed line'
Error retrieving bib data for  with yellow errorbands. The fit by \citet{madau2014sfrd: 'with yellow errorbands. the fit by \\citet{madau2014sfrd'
Retrieving document from  https://arxiv.org/e-print/2511.14216


...

 done.


E. Matthews  ->  E. Matthews  |  ['E. Matthews']


Issues with the citations
plugin pybtex.database.input.suffixes for suffix .tex not found
Retrieving document from  https://arxiv.org/e-print/2511.14733
extracting tarball to tmp_2511.14733...

 done.


P. Sudarshan  ->  P. Sudarshan  |  ['P. Sudarshan']
M. Flock  ->  M. Flock  |  ['M. Flock']
D. M. Fuksman  ->  D. M. Fuksman  |  ['D. M. Fuksman']


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.13828-b31b1b.svg)](https://arxiv.org/abs/2511.13828) | **The observed total star formation rate function up to z \sim 6: complementary UV and IR contributions and comparison with state-of-the-art galaxy formation models**  |
|| A. Traina, et al. -- incl., <mark>L. Xie</mark>, <mark>E. Schinnerer</mark> |
|*Appeared on*| *2025-11-19*|
|*Comments*| *14 pages, 8 figure, accepted for publication on A&A*|
|**Abstract**|            We investigate how the obscured IR-derived and the dust-corrected UV star formation rate functions (SFRFs) compare with each other, and with predictions from state-of-the-art theoretical models of galaxy formation and evolution. We derive the IR-SFRF from the ALMA A$^3$COSMOS survey, by converting the IR luminosity functions (IR-LFs) into SFRF after correcting for AGN contribution. Similarly, we obtain the UV SFRFs from literature UV LFs, corrected for dust-extinction. First, we fit the two SFRFs independently via a MCMC approach, then we combine them to obtain the first estimate of the total SFRF out to $z \sim 6$. Finally, we compare this SFRF with the predictions of a set of theoretical models. We derived the UV (dust-extinction corrected, from literature UV-LFs) and IR SFRFs (from Herschel and ALMA IR-LFs) at $0.5 < z < 6$ , finding that they are mostly complementary, covering different ranges in star formation rate (SFR$ < 10-100$ M$_{\odot}$yr$^{-1}$ for the UV-corrected and SFR$ > 100$ M$_{\odot}$yr$^{-1}$ for the IR). From the comparison of the total SFRF with model predictions we find an overall good agreement at $z < 2.5$, with increasing difference at higher redshifts, with all models missing the galaxies that are forming stars with the highest SFRs. We finally obtained the UV (dust-corrected), IR and total star formation rate densities (SFRDs), finding that there are no redshift ranges where UV and IR alone are able to reproduce the whole total SFRD.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.14289-b31b1b.svg)](https://arxiv.org/abs/2511.14289) | **Solar carbon abundance from 3D non-LTE modelling of the diagnostic lines of the CH molecule**  |
|| <mark>R. Hoppe</mark>, et al. -- incl., <mark>M. Bergemann</mark>, <mark>P. Eitner</mark> |
|*Appeared on*| *2025-11-19*|
|*Comments*| *Accepted by MNRAS, 22 pages*|
|**Abstract**|            Context. The spectral lines of the CH molecule are a key carbon (C) abundance diagnostic in FGKM-type stars. These lines are detectable in metal-rich and, in contrast to atomic C lines, also in metal-poor late-type stars. However, only 3D LTE analyses of the CH lines have been performed so far. Aims. We test the formation of CH lines in the solar spectrum, using for the first time, 3D Non-LTE (NLTE) models. We also aim to derive the solar photospheric abundance of C, using the diagnostic transitions in the optical (4218 - 4356 Å) and infrared (33025 - 37944 Å). Methods. We use the updated NLTE model molecule from Popa et al. (2023) and different solar 3D radiation-hydrodynamics model atmospheres. The models are contrasted against new spatially-resolved optical solar spectra, and the center-to-limb variation (CLV) of CH lines is studied. Results. The 1D LTE and 1D NLTE models fail to describe the line CLV, and lead to underestimated solar C abundances. The 3D NLTE modelling of diagnostic lines in the optical and IR yields a carbon abundance of A(C)=$8.52\pm0.07$ dex. The estimate is in agreement with recent results based on neutrino fluxes measured by Borexino. Conclusions. 3D NLTE modelling and tests on spatially-resolved solar data are essential to derive robust solar abundances. The analysis presented here focuses on CH, but we expect that similar effects will be present for other molecules of astrophysical interest.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.14360-b31b1b.svg)](https://arxiv.org/abs/2511.14360) | **Deep Andromeda JCMT-SCUBA2 Observations. The Submillimeter Maps and Giant Molecular Clouds**  |
|| <mark>S. Jiao</mark>, et al. -- incl., <mark>H. Beuther</mark>, <mark>J. d. Brok</mark>, <mark>F. Xu</mark> |
|*Appeared on*| *2025-11-19*|
|*Comments*| *24 pages, 14 figures. Accepted for publication in ApJS*|
|**Abstract**|            We have carried out unprecedentedly deep, nearly confusion-limited JCMT-SCUBA2 mapping observations on the nearest spiral galaxy, M31 (Andromeda). The 850 $\mu$m image with a $\sim$50 pc resolution yields a comprehensive catalog of 383 giant molecular clouds (GMCs) that are associated with the spiral arms. In addition, it unveiled a population of 189 compact inter-arm GMCs in M31, which are mostly unresolved or marginally resolved. The masses of all these GMCs are in the range of 2$\times$10$^4$ -- 6$\times$10$^6$ $M_{\odot}$; the sizes are in the range of 30--130 pc. They follow a mass-size correlation, $M$ $\propto$ $R_{c}$$^{2.5}$. The inter-arm GMCs are systematically less massive, more diffuse, colder, and have lower star-forming efficiency (SFE) than on-arm GMCs. Moreover, within individual spatially resolved on-arm and off-arm M31 GMCs, the SFE is considerably lower than the SFE in molecular clouds in main sequence and green valley galaxies. Follow-up investigations on M31 GMCs may provide clues for how star formation may be quenched in galactic environments. Finally, we reconstrained the dust opacity spectral index $\beta$ in the M31 galaxy by combining our new JCMT observations with archival Herschel and Planck data and found that the radial variation of $\beta$ may not be as large as was proposed by previous studies.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.14418-b31b1b.svg)](https://arxiv.org/abs/2511.14418) | **MINDS: The molecule-rich disc of the Herbig star HD 35929 revealed with JWST/MIRI**  |
|| T. Kaeufer, et al. -- incl., <mark>D. Gasman</mark>, <mark>T. Henning</mark>, <mark>G. Perotti</mark>, <mark>K. Schwarz</mark> |
|*Appeared on*| *2025-11-19*|
|*Comments*| *14 pages, 13 figures; accepted for publication in MNRAS*|
|**Abstract**|            Our knowledge of the chemical composition of the gas in the inner disc of intermediate-mass young stars is limited, due to the lack of suitable instrumentation. The launch of JWST has provided a significant improvement in our ability to probe gas in these inner discs. We analyse the gas composition and emitting conditions of the disc around HD 35929, a young intermediate-mass Herbig star, using MIRI/MRS data. Our goal is to constrain the chemistry and kinematics of the gas phase molecules detected in the inner disc. We use iSLAT to examine the observed molecular lines and DuCKLiNG to detect, fit, and analyse the molecular emission. We find gas phase H2O, CO, CO2, and OH in the disc, as well as HI recombination lines. Surprisingly, we also detect gas phase SiO in the fundamental v=1-0 vibrational band. We derive column densities and temperature ranges of the detected species, arising from the inner ~0.2 au, hinting towards a compact and very warm disc. The molecular column densities are much higher than found in lower mass T Tauri discs. In general, the molecular composition is consistent with an O-rich gas from which silicate-rich solids condense and the strong gas phase molecular line emission suggests a low dust opacity. The unexpected detection of gas phase SiO at the source velocity points to an incomplete condensation of rock forming elements in the disc, suggesting chemical disequilibrium and/or an underestimate of the gas kinetic temperature.         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.14733-b31b1b.svg)](https://arxiv.org/abs/2511.14733) | **Starlight-driven flared-staircase geometry in radiation hydrodynamic models of protoplanetary disks**  |
|| <mark>P. Sudarshan</mark>, et al. -- incl., <mark>M. Flock</mark>, <mark>D. M. Fuksman</mark> |
|*Appeared on*| *2025-11-19*|
|*Comments*| *17 pages, 16 figures, re-submitted to A&A after referee comments; abstract shortened for arXiv submission*|
|**Abstract**|            Protoplanetary disks observed in millimeter continuum and scattered light show a variety of substructures. Various physical processes in the disk could trigger such features -- one of which that has been previously theorized for passive disks is the thermal wave instability -- the flared disk may become unstable as directly illuminated regions puff up and cast shadows behind them. This would manifest as bright and dark rings, and a staircase-like structure in the disk optical surface. We provide a realistic radiation hydrodynamic model to test the limits of the thermal wave instability in irradiated disks. We carry out global axisymmetric 2D hydrostatic and dynamic simulations including radiation transport with frequency-dependent ray-traced irradiation and flux-limited diffusion (FLD). We found that starlight-driven shadows are most prominent in optically thick, slow cooling disks, shown by our models with high surface densities and dust-to-gas ratios of sub-micron grains of 0.01. We recover that thermal waves form and propagate inwards in the hydrostatic limit. In contrast, our hydrodynamic models show bumps and shadows within 30 au that converge to a quasi-steady state on several radiative diffusion timescales -- indicating a long-lived staircase structure. We find that existing thermal pressure bumps could produce and enhance this effect, forming secondary shadowing downstream. Hydrostatic models with self-consistent dust settling instead show a superheated dust irradiation absorption surface with a radially smooth temperature profile without staircases. We conclude that one can recover thermally induced flared-staircase structures in radiation hydrodynamic simulations of irradiated protoplanetary disks using flux-limited diffusion. We highlight the importance of modeling dust dynamics consistently to explain starlight-driven shadows.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.14216-b31b1b.svg)](https://arxiv.org/abs/2511.14216) | **Unveiling the Sources of X-ray Luminosity in DESI Galaxy Groups: Insights from the SRG/eROSITA All-Sky Survey**  |
|| Y. Zheng, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-11-19*|
|*Comments*| *29 pages, 14 figures, ApJ accepted*|
|**Abstract**|            We use the first eROSITA all-sky survey (eRASS1) to investigate the contributions of AGN and extended gas to the total X-ray luminosity ($L_X$) of galaxy groups with different halo masses ($M_h$) at different redshifts. The presence of AGN in their central galaxies is identified using multi-wavelength catalogs, including the X-ray counterparts, the ASKAP radio catalog, and the DESI spectroscopic measurements. We apply the stacking method to obtain sufficient statistics for the X-ray surface brightness profile and the $L_X$ for groups with different central AGN properties. We find that the X-ray groups exhibit the highest $L_X$, followed by groups with QSO, radio, BPT-AGN, and non-AGN centrals. Moreover, the $L_X$ of the $M_h \lesssim 10^{13}h^{-1}M_\odot$ groups is dominated by the central AGN, while the X-ray emission from extended gas tends to be more prominent in the $M_h \gtrsim 10^{13}h^{-1}M_\odot$ groups. In groups where the AGN play a major role in X-ray emission, the contribution from extended gas is minor, resulting in significant uncertainties concerning the extended X-ray emission. When the subset containing the X-ray detected counterparts is excluded, the extended gas component becomes easier to obtain. A correlation has been identified between the X-ray luminosity of the central AGN and extended gas. However, once we account for the positional offset, their correlation becomes less prominent. Currently, the results are not conclusive enough to confirm whether there is a connection between the AGN feedback and extended gas. However, they provide a new perspective on the feedback processes in the history of group assembly.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.14356-b31b1b.svg)](https://arxiv.org/abs/2511.14356) | **The SRG/eROSITA all-sky survey: X-ray scaling relations of galaxy groups and clusters in the western Galactic hemisphere**  |
|| M. E. Ramos-Ceja, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-11-19*|
|*Comments*| *13 pages, 18 figures. Submitted to A&A*|
|**Abstract**|            The soft X-ray telescope on board the Spectrum-Roentgen-Gamma (SRG) mission, eROSITA (extended ROentgen Survey with an Imaging Telescope Array), has produced the largest sample to date of galaxy groups and clusters detected via their intracluster/intragroup medium (ICM/IGrM) emission. Scaling relations between the intrinsic properties of these systems provide valuable insight into their formation and evolution. In this work, we investigate the scaling relations between key physical properties, such as soft band X-ray luminosity, temperature, gas mass, and the low-scatter mass proxy $Y_{\rm X}$, for the galaxy groups and clusters detected in the first eROSITA All-Sky Survey (eRASS1). Our analysis fully accounts for selection effects and the redshift evolution of the observable distributions. We construct a high-purity sample of $3061$ galaxy groups and clusters spanning the redshift range $0.05<z<1.07$ and mass range of $1.1\times10^{13}<M_{500}/$M$_{\odot}<1.6\times10^{15}$. This represents the largest sample to date used for scaling relation analysis. The selection function, derived from state-of-the-art simulations of the eROSITA sky, is rigorously incorporated into our modeling. We report best-fit parameters - normalization, slope, redshift evolution, and intrinsic scatter - for a set of scaling relations: $L_{\mathrm{X}}-T$, $L_{\mathrm{X}}-M_{\rm gas}$, $L_{\mathrm{X}}-Y_{\rm X}$, as well as the $M_{\rm gas}-T$ relation. Our best-fit models indicate that the slopes of the scaling relations deviate significantly from self-similar expectations, while the redshift evolution remains consistent with the self-similar model. The fits exhibit small statistical uncertainties, likely owing to the large sample size. Our results are in good agreement with previous observational studies that account for selection effects, as well as with simulations that incorporate non-gravitational physics.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.14407-b31b1b.svg)](https://arxiv.org/abs/2511.14407) | **ExoClock Project IV: A homogeneous catalogue of 620 updated exoplanet ephemerides**  |
|| A. Kokori, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2025-11-19*|
|*Comments*| *12 pages of text (104 in total, including the list of authors, acknowledgements, references and data tables), 7 figures, 8 tables. Submitted to ApJS (this version includes the first round of the reviewer's comments). The Data Release is available at: this https URL. Machine-readable versions of tables 7 and 8 are included the TeX Source*|
|**Abstract**|            The ExoClock project is an open platform aiming to monitor exoplanets by integrating observations from space and ground based telescopes. This study presents an updated catalogue of 620 exoplanet ephemerides, integrating 30000 measurements from ground-based telescopes (the ExoClock network), literature, and space telescopes (Kepler, K2 and TESS). The updated catalogue includes 277 planets from TESS which require special observing strategies due to their shallow transits or bright host stars. This study demonstrates that data from larger telescopes and the employment of new methodologies such as synchronous observations with small telescopes, are capable of monitoring special cases of planets. The new ephemerides show that 45% of the planets required an update while the results show an improvement of one order of magnitude in prediction uncertainty. The collective analysis also enabled the identification of new planets showing TTVs, highlighting the importance of extensive observing coverage. Developed in the context of the ESA's Ariel space mission, with the goal of delivering a catalogue with reliable ephemerides to increase the mission efficiency, ExoClock's scope and service have grown well beyond the remit of Ariel. The ExoClock project has been operating in the framework of open science, and all tools and products are accessible to everyone within academia and beyond, to support efficient scheduling of future exoplanet observations, especially from larger telescopes where the pressure for time allocation efficiency is higher (Ariel, JWST, VLT, ELT, Subaru etc). The inclusion of diverse audiences in the process and the collaborative mode not only foster democratisation of science but also enhance the quality of the results.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2511.13828/./sfrf_new/sfrf_combined_final_points.png', 'tmp_2511.13828/./sfrf_new/sfrf_combined_integral.png', 'tmp_2511.13828/./sfrf_new/sfrd_fit_madau_fujimoto_burgarella.png', 'tmp_2511.13828/./sfrf_new/sam_comparison_final.png']
copying  tmp_2511.13828/./sfrf_new/sfrf_combined_final_points.png to _build/html/
copying  tmp_2511.13828/./sfrf_new/sfrf_combined_integral.png to _build/html/
copying  tmp_2511.13828/./sfrf_new/sfrd_fit_madau_fujimoto_burgarella.png to _build/html/
copying  tmp_2511.13828/./sfrf_new/sam_comparison_final.png to _build/html/
exported in  _build/html/2511.13828.md
    + _build/html/tmp_2511.13828/./sfrf_new/sfrf_combined_final_points.png
    + _build/html/tmp_2511.13828/./sfrf_new/sfrf_combined_integral.png
    + _build/html/tmp_2511.13828/./sfrf_new/sfrd_fit_madau_fujimoto_burgarella.png
    + _build/html/tmp_2511.13828/./sfrf_new/sam_comparison_final.png
found figures ['tmp_2511.14289/./Figures/KPNO_diskcenter.png', 'tmp_2511.14289/./Fig

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\at}[1]{{\textcolor{magenta}{\bf[AT: #1]}}}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\cha}{\textit{Chandra}}$
$\newcommand{\XMM}{{XMM-\textit{Newton}}}$
$\newcommand{\NuSTAR}{\textit{NuSTAR}}$
$\newcommand{\Nu}{\textit{NuSTAR,}}$
$\newcommand{\bat}{{{\it Swift}-BAT}}$
$\newcommand\XSPEC{{\tt XSPEC}}$
$\newcommand\pexrav{{\tt pexrav}}$
$\newcommand\MYTorus{{\tt MYTorus}}$
$\newcommand\borus{{\tt borus02}}$
$\newcommand\bntorus{{\tt BNtorus}}$
$\newcommand\ngc{NGC 3081}$
$\newcommand\a{3 }$
$\newcommand\ad{A^320 }$</div>



<div id="title">

# The observed total star formation rate function up to $z \sim 6$: complementary UV and IR contributions and comparison with state-of-the-art galaxy formation models

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.13828-b31b1b.svg)](https://arxiv.org/abs/2511.13828)<mark>Appeared on: 2025-11-19</mark> -  _14 pages, 8 figure, accepted for publication on A&A_

</div>
<div id="authors">

A. Traina, et al. -- incl., <mark>L. Xie</mark>, <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:**            We investigate how the obscured IR-derived and the dust-corrected UV star formation rate functions (SFRFs) compare with each other, and with predictions from state-of-the-art theoretical models of galaxy formation and evolution. We derive the IR-SFRF from the ALMA A$^3$COSMOS survey, by converting the IR luminosity functions (IR-LFs) into SFRF after correcting for AGN contribution. Similarly, we obtain the UV SFRFs from literature UV LFs, corrected for dust-extinction. First, we fit the two SFRFs independently via a MCMC approach, then we combine them to obtain the first estimate of the total SFRF out to $z \sim 6$. Finally, we compare this SFRF with the predictions of a set of theoretical models. We derived the UV (dust-extinction corrected, from literature UV-LFs) and IR SFRFs (from Herschel and ALMA IR-LFs) at $0.5 < z < 6$ , finding that they are mostly complementary, covering different ranges in star formation rate (SFR$ < 10-100$ M$_{\odot}$yr$^{-1}$ for the UV-corrected and SFR$ > 100$ M$_{\odot}$yr$^{-1}$ for the IR). From the comparison of the total SFRF with model predictions we find an overall good agreement at $z < 2.5$, with increasing difference at higher redshifts, with all models missing the galaxies that are forming stars with the highest SFRs. We finally obtained the UV (dust-corrected), IR and total star formation rate densities (SFRDs), finding that there are no redshift ranges where UV and IR alone are able to reproduce the whole total SFRD.         

</div>

<div id="div_fig1">

<img src="tmp_2511.13828/./sfrf_new/sfrf_combined_final_points.png" alt="Fig5.1" width="50%"/><img src="tmp_2511.13828/./sfrf_new/sfrf_combined_integral.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** _ Upper panel:_ UV and IR SFRFs at different redshifts. Data points are plotted in cyan (UV) and red (IR), while the best fit is shown as blue and red curves for the UV and IR datasets, respectively. _ Lower panel:_ product between volume density ($\Phi$) and SFR in each redshift bin: this is the quantity that we integrate to obtain the SFRD. The UV (blue area) and the IR (red area) SFR density distributions are compared at different redshifts as function of the SFR. Both curves are normalize to have the peak of the UV at 1. (*fig:SFRF_points_fit*)

</div>
<div id="div_fig2">

<img src="tmp_2511.13828/./sfrf_new/sfrd_fit_madau_fujimoto_burgarella.png" alt="Fig4" width="100%"/>

**Figure 4. -** Fit to the total SFRD using the functional form by madau2014sfrd. The best fit to our data is the orange dashed line,  with yellow errorbands. The fit by \citet{madau2014sfrd is reported in black. A similar fit by [Fujimoto, et. al (2024)](https://ui.adsabs.harvard.edu/abs/2024ApJS..275...36F) is represented by the blue dotted line, while the green shaded area is the combined SFRD estimated by [Burgarella, et. al (2013)](https://ui.adsabs.harvard.edu/abs/2013A&A...554A..70B). In the upper-right corner, we display the best-fit values of the free parameters of Equation \ref{eq:sfrd_fit}.} (*fig:sfrd_fitted*)

</div>
<div id="div_fig3">

<img src="tmp_2511.13828/./sfrf_new/sam_comparison_final.png" alt="Fig7" width="100%"/>

**Figure 7. -** Observed SFRF compared with the prediction from simulations and SAMs. The yellow area represents the observed data used for the fit. The best-fit curve is reported in black. The purple dashed line represents the SFRF from the EAGLE simulation; the pink dashed curve is the result from the SIMBA simulation and the magenta line is from the IllustrisTNG \citep[][]{katsianis2017eagle, katsianis2021simba. The limegreen and green dotted curves are the prediction by [Hirschmann, De Lucia and Fontanot (2016)](https://ui.adsabs.harvard.edu/abs/2016MNRAS.461.1760H) and [Fontanot, et. al (2020)](https://ui.adsabs.harvard.edu/abs/2020MNRAS.496.3943F), from the GAEA SAM. Finally, the cyan dotted curves are the predictions by [Parente, et. al (2023)](https://ui.adsabs.harvard.edu/abs/2023MNRAS.521.6105P).} (*fig:LF_sam*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.13828"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Multi}{\texttt{MULTI}\xspace}$
$\newcommand{\MultiD}{\texttt{MULTI3D}\xspace}$
$\newcommand{\Dispatch}{\texttt{DISPATCH}\xspace}$
$\newcommand{\Must}{\texttt{M3DIS}\xspace}$
$\newcommand{\Turb}{\texttt{TURBOSPECTRUM}\xspace}$
$\newcommand{\Code}{\texttt{TUMULT}\xspace}$
$\newcommand{\Stagger}{\texttt{STAGGER}\xspace}$
$\newcommand{\Cobold}{\texttt{CO5BOLD}\xspace}$
$\newcommand{\Marcs}{\texttt{MARCS}\xspace}$
$\newcommand{\UOP}{\texttt{UOP}\xspace}$
$\newcommand{\Balder}{\texttt{BALDER}\xspace}$
$\newcommand{\Comment}[1]{{\xspace\color{Red}[#1]}\xspace}$
$\newcommand{\footnoterule}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Solar carbon abundance from 3D non-LTE modelling of the diagnostic lines of the CH molecule

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.14289-b31b1b.svg)](https://arxiv.org/abs/2511.14289)<mark>Appeared on: 2025-11-19</mark> -  _Accepted by MNRAS, 22 pages_

</div>
<div id="authors">

<mark>R. Hoppe</mark>, et al. -- incl., <mark>M. Bergemann</mark>, <mark>P. Eitner</mark>

</div>
<div id="abstract">

**Abstract:** _Context._ The spectral lines of the CH molecule are a key carbon (C) abundance diagnostic in FGKM-type stars. These lines are detectable in metal-rich and, in contrast to atomic C lines, also in metal-poor late-type stars.  However, only 3D LTE analyses of the CH lines have been performed so far. $\noindent$ _Aims._ We test the formation of CH lines in the solar spectrum, using for the first time, 3D Non-LTE (NLTE) models. We also aim to derive the solar photospheric abundance of C, using the diagnostic transitions in the optical ( $\SI{4218}$ - $\SI{4356}{\angstrom}$ ) and infrared ( $\SI{33025}$ - $\SI{37944}{\angstrom}$ ). $\noindent$ _Methods._ We use the updated NLTE model molecule from [Popa, et. al (2023)](https://ui.adsabs.harvard.edu/abs/2023A&A...670A..25P) and different solar 3D radiation-hydrodynamics model atmospheres. The models are contrasted against new spatially-resolved optical solar spectra, and the center-to-limb variation (CLV) of CH lines is studied. $\noindent$ _Results._ The 1D LTE and 1D NLTE models fail to describe the line CLV, and lead to underestimated solar C abundances. The 3D NLTE modelling of diagnostic lines in the optical and IR yields a carbon abundance of A(C)= $8.52\pm0.07$ dex. The estimate is in agreement with recent results based on neutrino fluxes measured by Borexino. $\noindent$ _Conclusions._ 3D NLTE modelling and tests on spatially-resolved solar data are essential to derive robust solar abundances. The analysis presented here focuses on CH, but we expect that similar effects will be present for other molecules of astrophysical interest.

</div>

<div id="div_fig1">

<img src="tmp_2511.14289/./Figures/KPNO_diskcenter.png" alt="Fig22.1" width="50%"/><img src="tmp_2511.14289/./Figures/KPNO_flux.png" alt="Fig22.2" width="50%"/>

**Figure 22. -** 1D LTE ($\Marcs$) and 3D NLTE ($\Must$) best-fits of optical diagnostic A-X transitions to the KPNO disk-center \protect ([Brault and Neckel 1987]())  and flux \protect ([Kurucz, et. al 1984](https://ui.adsabs.harvard.edu/abs/1984sfat.book.....K))  atlases. The legend labels show the corresponding C abundance. The vertical lines frame the windows used in the fitting-procedure. (*fig:CH_AX_KPNO*)

</div>
<div id="div_fig2">

<img src="tmp_2511.14289/./Figures/neutrino_fluxes.png" alt="Fig17" width="100%"/>

**Figure 17. -** Neutrino fluxes from global analysis of experimental neutrino measurements (black circles). The contours show predicted neutrino fluxes based on standard solar models with varying solar compositions. The black dashed tracks show the correlation between the fluxes and the solar core temperature. (*fig:neutrino_fluxes*)

</div>
<div id="div_fig3">

<img src="tmp_2511.14289/./Figures/IR_fits1.png" alt="Fig23" width="100%"/>

**Figure 23. -** Spacelab-3 \texttt{ATMOS} atlas observations \protect ([Farmer and Norton 1989](https://ui.adsabs.harvard.edu/abs/1989hra2.book.....F))  and 3D NLTE ($\Must$) line profile fits of infra-red vibration-rotation lines. The legend labels show the best-fit C abundance. The associated uncertainty is dominated by the uncertainty in the continuum placement. The vertical lines frame the windows used in the fitting-procedure. (*fig:IR_fits1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.14289"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mjypbm}{\mbox{mJy beam^{-1}}}$
$\newcommand{\bnote}[1]{{\bf\footnotesize \color{red}[HBL: #1]}}$
$\newcommand{\brevise}[1]{{\footnotesize \color{blue}[HBLrevised: #1]}}$
$\newcommand\green{#1 }$</div>



<div id="title">

# Deep Andromeda JCMT-SCUBA2 Observations. The Submillimeter Maps and Giant Molecular Clouds

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.14360-b31b1b.svg)](https://arxiv.org/abs/2511.14360)<mark>Appeared on: 2025-11-19</mark> -  _24 pages, 14 figures. Accepted for publication in ApJS_

</div>
<div id="authors">

<mark>S. Jiao</mark>, et al. -- incl., <mark>H. Beuther</mark>, <mark>J. d. Brok</mark>, <mark>F. Xu</mark>

</div>
<div id="abstract">

**Abstract:** We have carried out unprecedentedly deep, nearly confusion-limited JCMT-SCUBA2 mapping observations on the nearest spiral galaxy, M31 (Andromeda).The 850 $\mu$ m image with a $\sim$ 50 pc resolution yields a comprehensive catalog of 383 giant molecular clouds (GMCs) that are associated with the spiral arms.In addition, it unveiled a population of 189 compact inter-arm GMCs in M31, which are mostly unresolved or marginally resolved.The masses of all these GMCs are in the range of 2 $\times$ 10 $^4$ -- 6 $\times$ 10 $^6$ $M_{\odot}$ ; the sizes are in the range of 30--130 pc.They follow a mass-size correlation, $M$ $\propto$ $R_{c}^{2.5}$ .The inter-arm GMCs are systematically less massive, more diffuse, colder, and have lower star-forming efficiency (SFE) than on-arm GMCs.Moreover, within individual spatially resolved on-arm and off-arm M31 GMCs, the SFE is considerably lower than the SFE in molecular clouds in main sequence and green valley galaxies.Follow-up investigations on M31 GMCs may provide clues for how star formation may be quenched in galactic environments.Finally, we reconstrained the dust opacity spectral index $\beta$ in the M31 galaxy by combining our new JCMT observations with archival $_ Herschel_$ and $_ Planck_$ data and found that the radial variation of $\beta$ may not be as large as was proposed by previous studies.

</div>

<div id="div_fig1">

<img src="tmp_2511.14360/./Figures/mass.png" alt="Fig11.1" width="16%"/><img src="tmp_2511.14360/./Figures/td.png" alt="Fig11.2" width="16%"/><img src="tmp_2511.14360/./Figures/beta.png" alt="Fig11.3" width="16%"/><img src="tmp_2511.14360/./Figures/rc.png" alt="Fig11.4" width="16%"/><img src="tmp_2511.14360/./Figures/Sigma.png" alt="Fig11.5" width="16%"/><img src="tmp_2511.14360/./Figures/n.png" alt="Fig11.6" width="16%"/>

**Figure 11. -** 
Probability distribution functions (histograms) of Mass, dust temperature, $\beta$, R$_{c}$, $\Sigma_{dust}$ and $n_{\mbox{\scriptsize H}}$ for all the clouds in the catalog (black histograms), for on-arm clouds (orange histograms), and for inter-arm clouds (blue histograms).
The blue and orange dashed lines show the median values for off-arm clouds and on-arm clouds, respectively.
The typical values for these quantities are summarized in Table \ref{tab:sum}.
 (*fig:distribution*)

</div>
<div id="div_fig2">

<img src="tmp_2511.14360/./Figures/Ndust_rot_coolwarm_june.png" alt="Fig9.1" width="33%"/><img src="tmp_2511.14360/./Figures/Tdust_rot_coolwarm_june.png" alt="Fig9.2" width="33%"/><img src="tmp_2511.14360/./Figures/beta_rot_coolwarm_june.png" alt="Fig9.3" width="33%"/>

**Figure 9. -** 
The distribution of dust surface density (18$"$ angular resolution), temperature (18$"$ angular resolution), and $\beta$(37$"$ angular resolution) obtained from the iterative SED fits.
The detailed procedures are described in Section \ref{section:res_t_n}.
 (*fig:sed_maps*)

</div>
<div id="div_fig3">

<img src="tmp_2511.14360/./Figures/M31_obs.png" alt="Fig5" width="100%"/>

**Figure 5. -** 
The background color map shows the _Herschel_ 250 $\mu$m image  ([Fritz, et. al 2012](https://ui.adsabs.harvard.edu/abs/2012A&A...546A..34F), [Smith, et. al 2012](https://ui.adsabs.harvard.edu/abs/2012ApJ...756...40S)) . The field of view of the HASHTAG survey  ([Smith, et. al 2021](https://ui.adsabs.harvard.edu/abs/2021ApJS..257...52S))  is outlined in blue dot–dashed lines, while those of the JCMT large PI project and JCMT pilot project are shown in green solid and orange dashed lines, respectively.
 (*fig:m31_obs*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.14360"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# MINDS: The molecule-rich disc of the Herbig star HD 35929 revealed with JWST/MIRI

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.14418-b31b1b.svg)](https://arxiv.org/abs/2511.14418)<mark>Appeared on: 2025-11-19</mark> -  _14 pages, 13 figures; accepted for publication in MNRAS_

</div>
<div id="authors">

T. Kaeufer, et al. -- incl., <mark>D. Gasman</mark>, <mark>T. Henning</mark>, <mark>G. Perotti</mark>, <mark>K. Schwarz</mark>

</div>
<div id="abstract">

**Abstract:** Our knowledge of the chemical composition of the gas in the inner disc of intermediate-mass young stars is limited, due to the lack of suitable instrumentation. The launch of JWST has provided a significant improvement in our ability to probe gas in these inner discs.We analyse the gas composition and emitting conditions of the disc around HD 35929, a young intermediate-mass Herbig star, using MIRI/MRS data. Our goal is to constrain the chemistry and kinematics of the gas phase molecules detected in the inner disc.We use iSLAT to examine the observed molecular lines and DuCKLiNG to detect, fit, and analyse the molecular emission.We find gas phase $H_2$ O, CO, $CO_2$ , and OH in the disc, as well as HI recombination lines. Surprisingly, we also detect gas phase SiO in the fundamental $v$ =1-0 vibrational band. We derive column densities and temperature ranges of the detected species, arising from the inner $\sim0.2 \rm au$ , hinting towards a compact and very warm disc. The molecular column densities are much higher than found in lower mass T Tauri discs.In general, the molecular composition is consistent with an O-rich gas from which silicate-rich solids condense and the strong gas phase molecular line emission suggests a low dust opacity. The unexpected detection of gas phase SiO at the source velocity points to an incomplete condensation of rock forming elements in the disc, suggesting chemical disequilibrium and/or an underestimate of the gas kinetic temperature.

</div>

<div id="div_fig1">

<img src="tmp_2511.14418/./figures/163_mol_contribution_zoom_in_plot.png" alt="Fig3" width="100%"/>

**Figure 3. -** Continuum subtracted MIRI spectrum, using the 4.9 - 25 $\mu$m wavelength range, but excluding the 10 $\mu$m silicate band region between 9.0 and 13.5 $\mu$m (excluded region marked in grey). Overplotted are the cumulative fluxes from $H_2$O, CO, SiO, $CO_2$, and \ce{OH} from the median probability model. A few unfitted atomic lines and the non-detection of \ce{^{13}CO2} are labelled. The apparent feature around $18.55 \rm \mu m$ can be traced back to a bad pixel artifact. (*fig:mol-fit*)

</div>
<div id="div_fig2">

<img src="tmp_2511.14418/./figures/163_mol_contribution_plot_sio_emission_compare.png" alt="Fig5" width="100%"/>

**Figure 5. -** Residual in the wavelength range dominated by SiO of the full fit compared to a water-only fit. (*fig:mol-sio-zoom*)

</div>
<div id="div_fig3">

<img src="tmp_2511.14418/./figures/163_molecular_conditions_contour_version.png" alt="Fig6" width="100%"/>

**Figure 6. -** Probability distribution of the molecular column densities and temperatures derived from the continuum subtracted MIRI spectrum. The colours show the emissions for the molecules listed in the legend, while the contours denote the region with the highest contribution to the respective molecular fluxes. (*fig:mol-con*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.14418"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\reff}[1]{\textbf{#1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# A search for transit timing variations in the transiting hot Jupiter systems HIP 65, NGTS-6, NGTS-10 and WASP-173

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.14634-b31b1b.svg)](https://arxiv.org/abs/2511.14634)<mark>Appeared on: 2025-11-19</mark> -  _13 pages, 3 figures_

</div>
<div id="authors">

A. W. Griffiths, et al.

</div>
<div id="abstract">

**Abstract:** Hot Jupiters are Jupiter-mass planets with orbital periods of less than ten days. Their short orbital separations make tidal dissipation within the stellar host especially efficient, potentially leading to a measurable evolution of the orbit. One possible manifestation of this is orbital decay, which presents itself observationally through variations in the orbital period and thus times of transit. Here we select four promising exoplanetary systems for detecting this effect: HIP 65, NGTS-6, NGTS-10 and WASP-173. We present 33 new transit light curves taken with the 1.54 m Danish Telescope, and analyse these alongside photometric data from the Transiting Exoplanet Survey Satellite and transit timing data from the literature. We construct two ephemeris models for each target: a linear ephemeris and a shrinking orbital period due to tidal decay. The linear ephemeris is preferred for all four models -- the highest significance for the quadratic ephemeris is 2 $\sigma$ for WASP-173. We compare these results to theoretical predictions for tidal dissipation of gravity waves in radiation zones, and find that wave breaking is predicted only in WASP-173, making rapid decay plausible in this system but unclear in the other three. The sensitivity of transit timings to orbital decay depends on the square of the time interval covered by available observations, so our results establish a useful baseline against which future measurements can be compared. NGTS-6 and NGTS-10 are important objects for future study as they are in the first field to be observed by the upcoming PLATO mission.

</div>

<div id="div_fig1">

<img src="tmp_2511.14634/./AllResidPlot.png" alt="Fig2" width="100%"/>

**Figure 2. -** Linear ephemeris residuals plotted against the number of cycles from the median time. The solid line at $y=0$ represents the linear model, where the shaded grey region is its 1$\sigma$ error. The dashed line represents the quadratic model in terms of the linear residuals. Blue points denote new transit times from the Danish Telescope. Green points denote TESS transits. Red points denote transit times from the literature. Orange points denote times from the ETD. (*fig resid*)

</div>
<div id="div_fig2">

<img src="tmp_2511.14634/./AllFig.png" alt="Fig1" width="100%"/>

**Figure 1. -** New transit light curves for HIP 65 A, NGTS-6, NGTS-10 and WASP-173 A taken with the Danish Telescope. Data points are displayed in black with their errorbars. Fits are plotted in red. The dates of observation are displayed above each plot. (*fig:fig1*)

</div>
<div id="div_fig3">

<img src="tmp_2511.14634/./AllFig.png" alt="Fig3" width="100%"/>

**Figure 3. -** Phase-folded transit light curves for HIP 65 A, NGTS-6, NGTS-10 and WASP-173 A taken from TESS. Data points are displayed in black with their errorbars. Fits are plotted in red. The TESS sectors are displayed above each plot. 1/2 refers to the first half of a sector. 2/2 refers to the second half of a sector. (*fig:tess*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.14634"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# A socio-demographic study of the exoplanet direct imaging community 2

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.14645-b31b1b.svg)](https://arxiv.org/abs/2511.14645)<mark>Appeared on: 2025-11-19</mark> -  _22 pages, 8 figures_

</div>
<div id="authors">

L. Leboulleux, et al. -- incl., <mark>E. Matthews</mark>

</div>
<div id="abstract">

**Abstract:** Recognizing and addressing under-representation, exclusion, and problematic behavior, including discrimination, within astronomy and astrophysics are crucial. Therefore, in 2019, a survey was conducted at the Spirit of Lyot conference to evaluate the socio-demographics and well-being of the exoplanet and disk imaging community. In this paper, we present the results of a second survey, conducted as a followup at the 2022 Spirit of Lyot conference held in Leiden, the Netherlands. Its objective was to 1) improve the evaluation of our community, both in terms of questions of interest and diversity, and 2) monitor the evolution of the metrics since the 2019 socio-demographic study. This new survey was sent to all participants in Spirit of Lyot and received $96$ responses, equivalent to a participation rate of $44\%$ . It measured the visibility of respondents at conferences, their recognition through publications and projects, their experience of disrespect as victims or witnesses, their experience of inappropriate behaviors also as victims or witnesses, and their identification as allies of minorities in the field. These topics were studied with respect to job position, expatriation/non-expatriation, gender, belonging to another under-represented group (in terms of ethnicity, disability, or sexual orientation), and parenthood. Overall, comparing the survey respondent panel with the conference attendee information shows that survey respondents were more likely to be part of traditionally more-discriminated categories (women and non-permanent researchers, etc.). The survey also reveals biases against non-permanent researchers in terms of visibility (less invitation to Scientific Organizing Committees, less talks for PhD students). Regarding disrespect and inappropriate behaviors, the results are concerning: for instance, women and non-binary people are the most exposed to inappropriate behaviors, and are more likely to hold non-permanent positions. Considering these categories together, $36\%$ of female and non-binary PhD students (respectively $17\%$ of male PhD students) have experienced situations of inappropriate behaviors over the last $2.5$ years. There is also a clear trend between having experienced disrespect or inappropriate behavior and noticing that such a situation is happening. On a more positive note, most people ( $82\%$ ), from all categories, are willing to learn about alliance. This paper also provides recommended practices to improve the well-being and visibility of under-represented groups in astronomy.

</div>

<div id="div_fig1">

<img src="tmp_2511.14645/./Sec4_General_1.png" alt="Fig4.1" width="5%"/><img src="tmp_2511.14645/./Sec4_General_2.png" alt="Fig4.2" width="5%"/><img src="tmp_2511.14645/./Sec4_General_3.png" alt="Fig4.3" width="5%"/><img src="tmp_2511.14645/./Sec4_Position_1.png" alt="Fig4.4" width="5%"/><img src="tmp_2511.14645/./Sec4_Position_2.png" alt="Fig4.5" width="5%"/><img src="tmp_2511.14645/./Sec4_Position_3.png" alt="Fig4.6" width="5%"/><img src="tmp_2511.14645/./Sec4_Expatriation_1.png" alt="Fig4.7" width="5%"/><img src="tmp_2511.14645/./Sec4_Expatriation_2.png" alt="Fig4.8" width="5%"/><img src="tmp_2511.14645/./Sec4_Expatriation_3.png" alt="Fig4.9" width="5%"/><img src="tmp_2511.14645/./Sec4_Gender_1.png" alt="Fig4.10" width="5%"/><img src="tmp_2511.14645/./Sec4_Gender_2.png" alt="Fig4.11" width="5%"/><img src="tmp_2511.14645/./Sec4_Gender_3.png" alt="Fig4.12" width="5%"/><img src="tmp_2511.14645/./Sec4_UnderRep_1.png" alt="Fig4.13" width="5%"/><img src="tmp_2511.14645/./Sec4_UnderRep_2.png" alt="Fig4.14" width="5%"/><img src="tmp_2511.14645/./Sec4_UnderRep_3.png" alt="Fig4.15" width="5%"/><img src="tmp_2511.14645/./Sec4_Father_1.png" alt="Fig4.16" width="5%"/><img src="tmp_2511.14645/./Sec4_Father_2.png" alt="Fig4.17" width="5%"/><img src="tmp_2511.14645/./Sec4_Father_3.png" alt="Fig4.18" width="5%"/>

**Figure 4. -** 
Expertise recognition through inclusion in publications and projects, since January 2020: (left) percentage of people, per category, having felt unfairly absent from a co-author list, (center) percentages of people, per category, having felt unfairly present in a co-author list, (right) percentages of people, per category, having felt left out from a publication or project. (*fig:Sec4_Fig1*)

</div>
<div id="div_fig2">

<img src="tmp_2511.14645/./Section2-Demographics-10.png" alt="Fig1" width="100%"/>

**Figure 1. -** 
The `leaky pipeline', demonstrating the drop-out rates as a function of increasing career stage, is illustrated with data from the current survey. In addition to showing this real phenomenon in astronomy, it indicates a correlation between job position and gender in the interpretation of the outcomes of the survey later in this paper. (*fig:Sec2_Fig3*)

</div>
<div id="div_fig3">

<img src="tmp_2511.14645/./Annexe_1.png" alt="Fig2.1" width="33%"/><img src="tmp_2511.14645/./Annexe_2.png" alt="Fig2.2" width="33%"/><img src="tmp_2511.14645/./Annexe_3.png" alt="Fig2.3" width="33%"/>

**Figure 2. -** 
Outcome regarding the research expertise: (left) percentages of respondents, per category, who have felt unfairly absent from a list of co-authors, (center) percentages of respondents, per category, who declare having experienced disrespect (right) percentages of respondents, per category, who declare having experienced a situation of inappropriate behavior. (*fig:Annexe*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.14645"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kruti}[1]{\textcolor{ForestGreen}{#1}}$
$\newcommand{\alex}[1]{\textcolor{BurntOrange}{#1}}$
$\newcommand{\mario}[1]{\textcolor{red}{#1}}$
$\newcommand{\david}[1]{\textcolor{JungleGreen}{#1}}$
$\newcommand{\editor}[1]{#1}$
$\newcommand{\referee}[1]{#1}$
$\newcommand{\alexreplace}[2]{\alex{\st{#1}{~#2}}}$
$\newcommand{\alexnote}[1]{\alex{\textbf{#1}}}$
$\newcommand{\davidnote}[1]{\david{\textbf{#1}}}$
$\newcommand{\DP}[2]{\frac{\partial{#1}}{\partial{#2}}}$
$\newcommand{\D}[2]{\frac{\text{d}{#1}}{\text{d}{#2}}}$
$\newcommand{\Int}{\int\limits}$
$\newcommand{\rbeg}{r_\text{beg}}$
$\newcommand{\phistar}{\Phi_\star}$
$\newcommand{\Mstar}{M_\star}$
$\newcommand{\Msun}{\text{M}_{\odot}}$
$\newcommand{\Rsun}{\text{R}_{\odot}}$
$\newcommand{\Fstar}{F_\star}$
$\newcommand{\Rstar}{R_\star}$
$\newcommand{\Tstar}{T_\star}$
$\newcommand{\Lstar}{L_\star}$
$\newcommand{\Etot}{E_\text{tot}}$
$\newcommand{\Erad}{E_\text{rad}}$
$\newcommand{\Frad}{F_\text{rad}}$
$\newcommand{\bmFrad}{\bm{F}_\text{rad}}$
$\newcommand{\arad}{a_\text{R}}$
$\newcommand{\kp}{\kappa_\text{P}}$
$\newcommand{\kr}{\kappa_\text{R}}$
$\newcommand{\knu}{\kappa_\nu}$
$\newcommand{\G}{\text{G}}$
$\newcommand{\sigsb}{\sigma_\text{SB}}$
$\newcommand{\csiso}{c_\mathrm{s,iso}}$
$\newcommand{\cs}{c_\mathrm{s}}$
$\newcommand{\csadb}{c_\mathrm{s}^\mathrm{ad}}$
$\newcommand{\OmegaK}{\Omega_\mathrm{K}}$
$\newcommand{\mean}[1]{\langle{#1} \rangle}$
$\newcommand{\cv}{c_\mathrm{v}}$
$\newcommand{\Rgas}{\mathcal{R}}$
$\newcommand{\St}{\text{St}}$
$\newcommand{\edg}{\epsilon_\text{dg}}$</div>



<div id="title">

# Starlight-driven flared-staircase geometry in radiation hydrodynamic models of protoplanetary disks

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.14733-b31b1b.svg)](https://arxiv.org/abs/2511.14733)<mark>Appeared on: 2025-11-19</mark> -  _17 pages, 16 figures, re-submitted to A&A after referee comments; abstract shortened for arXiv submission_

</div>
<div id="authors">

<mark>P. Sudarshan</mark>, et al. -- incl., <mark>M. Flock</mark>, <mark>D. M. Fuksman</mark>

</div>
<div id="abstract">

**Abstract:** Protoplanetary disks observed in millimeter continuum and scattered light show a variety of substructures. While embedded planets are a common explanation, during the early planet formation phase, various physical processes in the disk could also trigger such features. One such possibility that has been previously theorized for passive disks is the the thermal wave instability or the stellar irradiation instability---the flared disk may become unstable as directly illuminated regions puff up and cast shadows behind them. This would manifest as $\referee{bright and dark rings}$ , and a staircase-like structure in the disk optical surface. We provide a realistic radiation hydrodynamic model to test the limits of the thermal wave instability in starlight-heated protoplanetary disks. We make quantitative comparisons to existing results in literature from simpler linear theory and 1D models to moment transfer methods, elucidating the importance of correct numerical treatment for this problem. We carry out global axisymmetric 2D hydrostatic and dynamic simulations including radiation transport with frequency-dependent ray-traced irradiation and flux-limited diffusion (FLD). We vary dust-to-gas ratios and surface densities. We also highlight the role of small grains and dust settling with the first radiation hydrostatic dust models to study starlight-driven shadowing. We found that starlight-driven shadows are most prominent in optically thick, slow cooling disks, shown by our models with high surface densities and dust-to-gas ratios of sub-micron grains $\edg=0.01$ . We recover that $\referee{thermal waves}$ form and propagate inwards in the hydrostatic limit. In contrast, our hydrodynamic models show $\referee{bumps and shadows}$ within $30$ au that converge to a quasi-steady state on several $\referee{radiative}$ diffusion timescales---indicating a long-lived staircase structure. $\referee{We find that existing thermal pressure bumps could produce and enhance this effect, forming secondary structure due to starlight-driven shadowing downstream.}$ Hydrostatic models with self-consistent dust settling instead show a superheated dust irradiation absorption surface with a radially smooth temperature profile $\referee{without staircases.}$ We conclude that we can recover thermally induced flared-staircase structure in radiation hydrodynamic simulations of irradiated protoplanetary disks using the flux-limited diffusion method. We find that the $\referee{shadowing effect}$ is sensitive to the dust content in the disk. We highlight the importance of modeling dust dynamics consistently to explain starlight-driven shadows.

</div>

<div id="div_fig1">

<img src="tmp_2511.14733/./timeevoldyn.png" alt="Fig4" width="100%"/>

**Figure 4. -** $\referee${Top: }Time evolution of the temperature profile at the disk midplane against the background power law. We see here that the bumps are not moving inwards chaotically, and while we lose the outer bump, the inner two stabilize at about $t = 400$ orbits. The first bump grows in amplitude, and as a result casts a steeper shadow behind, moving the second shadow further than where it started. The $\referee${dashed} lines (the first corresponds to the opacity transition) indicate the final locations of the temperature bumps. $\referee${Bottom: Time evolution of the radial temperature gradient. In shadowed regions the profile gets less steep, but overall the "bumps" are not strong enough to induce a sign flip.} (*fig:dyn-time-evol*)

</div>
<div id="div_fig2">

<img src="tmp_2511.14733/./opacfig.png" alt="Fig2" width="100%"/>

**Figure 2. -** Top panel: Frequency-dependent dust absorption opacities (per gram of dust, $\text{g}_\text{dust}$) from tabulated values  ([Krieger and Wolf 2020](https://ui.adsabs.harvard.edu/abs/2020A&A...635A.148K), [Krieger and Wolf 2022](https://ui.adsabs.harvard.edu/abs/2022A&A...662A..99K)) . The blue points denote $\referee${132 frequencies logarithmically sampled in the range $\nu \in [1.5 \times 10^{11}, 6 \times 10^{15}]$ Hz} for the computation of irradiation flux in our models. The black line indicates the black-body function and the orange line is the Planck opacity for $\Tstar = 4000 $K \citep[figure similar to][]{2010Kuiper}. Bottom panel: Temperature–dependent Planck and Rosseland mean opacities used for radiation transfer. $\referee${The local maximum ($\approx$ 250 K) and the subsequent dip in the mean opacities is due to the $10 \mu$m silicate feature.} (*fig:opacfig*)

</div>
<div id="div_fig3">

<img src="tmp_2511.14733/./tempcomp_rmc.png" alt="Fig7" width="100%"/>

**Figure 7. -** Comparison of 1D temperature profiles of the hydrostatic models at the end of the simulation for two different radiation methods --- flux-limited diffusion (FLD, used in this work) and moment transfer \citep[M1, ][]{2022FuksmanKlahr}. Overall we see very good agreement with temperatures with FLD and M1, with the optically thin atmosphere being captured quite well. The black dotted line shows the analytical solution for the temperature profile in the optically thin atmosphere  ([Chiang and Goldreich 1997](https://ui.adsabs.harvard.edu/abs/1997ApJ...490..368C)) . The midplane temperatures are also in good agreement, with some minor deviations. The M1 method shows higher amplitude bumps than the FLD method. $\referee${We also include the RADMC-3D calculations corresponding to our FLD model, these correspond to the green lines.} (*fig:tempcomp*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.14733"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

129  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

15  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

10  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
